# Escaneando

Esta es la segunda fase del bloque de reconocimiento. Tras haber recopilado toda la información posible sobre el objetivo, en esta fase intentaremos descubrir qué equipos del cliente tiene expuestos en internet (escaneo de red), qué servicios tienen habilitados esos equipos (escaneo de puertos) y si esos equipos y servicios habilitados tienen vulnerabilidades conocidas (escaneo de vulnerabilidades). El objetivo de esta fase es utilizar toda la información que hemos encontrado en la fase anterior para detectar vectores de ataque y vías de entrada en la red que se este auditando.

### Metodología

Se recomienda seguir usando [cherrytree](https://www.giuspen.com/cherrytree/) para ir guardando las evidencias.

1. <a style="color:#000;text-decoration:none;cursor:pointer" href="#intro">Breve introducción informal</a>
  * ICMPv4
  * Cabecera del protocolo TCP
<br><br>
2. <a style="color:#000;text-decoration:none;cursor:pointer" href="#escaneando-red">Escaneando la red para descubrir equipos activos</a>
  * Detectar si un equipo esta activo usando ping
  * Ping Sweep/ICMP Sweep detectar los equipos activos en una red
  * Ping Sweep/ICMP Sweep usando nmap
  * Detectar equipos activos usando netdiscover
  * Descubriendo equipos activos por TCP usando hping3
<br><br>
2. <a style="color:#000;text-decoration:none;cursor:pointer" href="#detectando-servicios">Escaneando los puertos de los equipos activos</a>
  * Equipo para realizar escaneos de pruebas (scanme.nmap.org)
  * Escaneando equipos desactivando el host discovery
  * Escaneo con handshake completo (fiable pero ruidoso)
  * Escaneo con medio handshake (SYN Scan)
  * Escaneo detectando la versión de los servicios
  * Escaneo de medio handshake detectando la versión de los servicios
  * Escaneo de servicios en puertos UDP
  * Detectando el sistema operativo
    * nmap
    * p0f
  * Escaneo agresivo con nmap
  * Escaneo ACK para descubrir dispositivos de seguridad
    * nmap
    * hping3
  * Escaneo con nmap a través de TOR usando proxychains
<br><br>

3. <a style="color:#000;text-decoration:none;cursor:pointer" href="#detectando-vulnerabilidades">Escaneando en busca de vulnerabilidades conocidas</a>
  * Breve introducción informal sobre los escáneres de vulnerabilidades
<br><br>

### Referencias usadas

1. Páginas web
  * [OSI](https://es.wikipedia.org/wiki/Modelo_OSI)
  * [Nmap examples](https://nmap.org/man/es/man-port-scanning-techniques.html)
  * [Nmap Host discovery](https://nmap.org/book/man-host-discovery.html)
  * [Nmap port scanning techniques](https://nmap.org/man/es/man-port-scanning-techniques.html)
  * [RFC 792 - ICMPv4](https://tools.ietf.org/html/rfc792)
  * [RFC 793 - TCP](https://tools.ietf.org/html/rfc793)

<a name="intro"></a>

### 1. Breve introducción informal

#### ICMPv4

Debido a los errores que pueden existir en una red de computadores, necesitamos un modo de saber el estado de los equipos que componen esa red y obtener información para solucionar los problemas que puedan surgir. Ahí es donde entra Internet Control Message Protocol (ICMP). ICMP es un protocolo que nos va a permitir hacer consultas a distintos equipos de una red para conocer su estado. Existen dos versiones de este protocolo:

  * ICMPv4
  * ICMPv6

Nos centraremos en [ICMPv4](https://tools.ietf.org/html/rfc792) porque es el que necesitamos para el contenido del curso. El uso más común del protocolo ICMP es detectar si un equipo esta activo en una red. Lo que coloquialmente llamamos hacer un ping a un equipo. ICMP es un protocolo que opera en la [capa 3](https://es.wikipedia.org/wiki/Capa_de_red) del modelo [OSI](https://es.wikipedia.org/wiki/Modelo_OSI) que funciona enviando peticiones de distintos tipos a los equipos que hay en una red y analizando los tipos de respuesta que recibe. Los tipos de peticiones y de respuesta que se pueden enviar o recibir están unívocamente identificados por un número que podemos consultar en la web de [IANA](https://www.iana.org/assignments/icmp-parameters/icmp-parameters.xhtml). Por ejemplo, el tipo de petición para saber si un equipo está activo en una red es el número 8. Si lanzamos un ping (petición ICMP tipo 8) a un equipo en una red donde el protocolo ICMP esté habilitado y no nos responde, sabremos que el equipo no esta activo. Sin embargo, si el equipo nos responde al ping con un paquete ICMP de tipo 0, sabremos que el equipo está activo en la red.

#### Cabecera del protocolo TCP

Si accedemos a la especificación del protocolo [TCP](https://tools.ietf.org/html/rfc793#page-15) podremos ver la estructura de la cabecera adjunta a todo paquete del protocolo TCP:

<img src="img/tcp_1.png" style="float:left">
<div style="clear:both"></div><br>

Los 6 bits marcados en amarillo corresponden con las flags del protocolo TCP. Estos 6 bits, llamados flags en la cabecera del protocolo TCP, gobiernan la transmisión de datos que se produce en una comunicación a través del protocolo TCP. Con estas flags podemos iniciar, mantener o interrumpir cualquier transmissión a través del protocolo TCP. El detalle de cada flag esta detallado en el temario oficial. Sólo mencionar que, los sistemas operativos basados en [UNIX](https://es.wikipedia.org/wiki/Unix) como GNU/Linux, implementan la pila TCP/IP de acuerdo a esta especificación, lo cuál nos será útil conocerla a la hora de realizar escaneos. Podremos activar y desactivar estos bits llamados flags con el fin de realizar distintos tipos de escaneos. Los sistemas operativos windows no implementan su pila de acuerdo a esta especificación.

<a name="escaneando-red"></a>

### 2. Escaneando la red para descubrir equipos activos

En la fase anterior encontramos el rango de red asociado a la organización u objetivo que se este auditando. Ahora trataremos de descubrir que equipos hay activos en esa red.

#### Detectar si un equipo esta activo usando ping

Podemos usar el comando ping para saber si un equipo esta activo en nuestra red:

<img src="img/ping_1.png" style="float:left">
<div style="clear:both"></div><br>

o en internet:

<img src="img/ping_2.png" style="float:left">
<div style="clear:both"></div><br>

*la opción **-c** es para indicar el número de pings (peticiones ICMP de tipo 8) que queremos enviar*

#### Ping Sweep/ICMP Sweep detectar los equipos activos en una red

Ping Sweep, ICMP Sweep o un barrido de red es hacer un ping a cada equipo de una red para saber si están activos o no. Aunque se podría hacer con un bucle for y analizar la respuesta de cada equipo:

<img src="img/ping_sweep_1.png" style="float:left">
<div style="clear:both"></div><br>

Existen programas que nos van a facilitar esta tarea como veremos a continuación.

#### Ping Sweep/ICMP Sweep usando nmap

Una forma muy sencilla de hacer ping sweep es con nmap. Para ello usamos nmap con la opción *-sn* para deshabilitar el escaneo de puertos y realizar únicamente un ping sweep:

<img src="img/ping_sweep_nmap.png" style="float:left">
<div style="clear:both"></div><br>

como podemos ver, si el protocolo ICMP esta activo, nos detecta todos los equipos activos en esa red.

#### Detectar equipos activos usando netdiscover

Otro programa para detectar los equipos de una red es netdiscover. Esta herramienta funciona esnifando el tráfico ARP. Lo lanzamos usando el comando:

<img src="img/netdiscover_1.png" style="float:left">
<div style="clear:both"></div><br>

y podremos ver los resultados en pantalla:

<img src="img/netdiscover_2.png" style="float:left">
<div style="clear:both"></div><br>

#### Descubriendo equipos activos por TCP usando hping3

Si el protocolo ICMP no estuviese activado en la red que estemos trabajando, podemos usar el protocolo TCP para saber si un equipo esta activo en la red. Un programa con el que podemos hacer esto de forma sencilla es hping3. Hping3 nos va a permitir mandar paquetes TCP con ciertas flags activadas. Por ejemplo, podemos mandar un paquete TCP con la flag *SYN* activada usando la opción *-S*:

<img src="img/hping3_1.png" style="float:left">
<div style="clear:both"></div><br>

*la opción **-c** es para indicar el número de paquetes que queremos enviar*

<a name="detectando-servicios"></a>

### 3. Escaneando los puertos de los equipos activos

Una vez que hemos descubierto los equipos que están activos en una red, intentaremos averiguar qué puertos y qué servicios tienen activos esos equipos.

#### Equipo para realizar escaneos de pruebas (scanme.nmap.org)

El proyecto Nmap ha puesto un equipo en internet para que podamos practicar con nmap de forma responsable. El nombre de dominio es [scanme.nmap.org](http://scanme.nmap.org/):

<img src="img/nmap_equipo_para_practicar.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneando equipos desactivando el host discovery

Nmap, por defecto, detecta los equipos activos en una red lanzando ping (peticiones ICMP de tipo 8). Si el equipo:

  * responde al ping con un paquete ICMP de tipo 0, lo da como activo en la red
  * si no responde al ping lo da como no activo

Si nmap da un equipo por no activo, no realizará más pruebas como un escaneo de puertos. Esto puede ocasionar que, si el protocolo ICMP no esta activado, se den como no activos equipos que si lo están. Por ello, nmap tiene una opción que le podemos pasar para que, en vez de descubrir qué equipos están activos usando ping, directamente dé todos los equipos como activos. De esta forma, podremos escanear los puertos de equipos que tengan bloqueados el tráfico ICMP. Esta opción es *-Pn* y quiere decir (Ping no).

#### Escaneo con handshake completo (fiable pero ruidoso)

Una de las formas que hay de conocer si un puerto TCP esta abierto es realizar el handshake completo. Si podemos realizar el handshake, entonces daremos el puerto como abierto. Es decir, se da el puerto como abierto si:

  1. Enviamos un paquete con la SYN flag activada
  2. Se recibe como respuesta un paquete con las flags SYN y ACK activadas
  3. Enviamos un packete con la flag ACK activada para establecer la conexión
  4. Cortamos la comunicación enviando un paquete con la RST flag activada

Se dice que es ruidoso porque, normalmente, si realizamos un handshake completo, el servidor guardará en sus logs nuestra conexión. La opción para realizar un escaneo con handshake completo es *-sT*. En este caso lanzamos un escaneo contra todos los puertos:

<img src="img/nmap_st_scan.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo con medio handshake (SYN Scan)

En este tipo de escaneo de puertos del protocolo TCP se hace medio handshake:

  1. Se envia un paquete con la SYN flag activada
  2. Si se recibe como respuesta un paquete con las flags SYN y ACK activadas, se da como activo
  3. Se corta el handshake enviando un paquete con la RST flag activada
  
Muchos servidores no guardarán en sus logs nuestra conexión ya que no se ha producido un handshake completo. Aunque si el la red que estemos auditando cuenta con dispositivos de seguridad como [IDS](https://es.wikipedia.org/wiki/Sistema_de_detección_de_intrusos), nuestra conexión quedará registrada. La opción que usaremos para lanzar un escaneo de este tipo es *-sS*. En este caso, vamos a lanzarlo contra puertos específicos:

<img src="img/nmap_syn_scan.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo detectando la versión de los servicios

Para detectar la versión de los servicios (programas) que hay corriendo en cada puerto podemos usar la opción *-sV*. Lo que hará nmap es intentar obtener el [banner](https://en.wikipedia.org/wiki/Banner_grabbing) de cada servicio:

<img src="img/nmap_sv_scan_1.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo de medio handshake detectando la versión de los servicios

Podemos combinar el escaneo SYN Scan con la detección de servicios pasando ambas opciones, *-sS* y *-sV* al lanzar nmap:

<img src="img/nmap_sv_scan_2.png" style="float:left">
<div style="clear:both"></div><br>

o usando la versión abreviada *-sSV*:

<img src="img/nmap_sv_scan_3.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo de servicios en puertos UDP

La opción para descubrir servicios en puertos del protocolo UDP es *-sU*:

<img src="img/nmap_su_scan.png" style="float:left">
<div style="clear:both"></div><br>

#### Detectando el sistema operativo

Debido a la implementación que cada sistema operativo hace la pila [TCP/IP](https://en.wikipedia.org/wiki/TCP/IP_stack_fingerprinting), en muchas ocasiones, se puede determinar el sistema operativo de un equipo remoto enviando paquetes y analizando valores en los paquetes de respuesta como el ISN, el tamaño de ventana o el TTL. Podemos usar la opción *-O* para detectar el sistema operativo usando nmap:

<img src="img/nmap_os_scan.png" style="float:left">
<div style="clear:both"></div><br>

Otro programa que podemos utilizar para averiguar el sistema operativo es [p0f](http://lcamtuf.coredump.cx/p0f3/). P0f detecta el sistema operativo de forma pasiva esnifando tráfico. Para usarlo, primero debemos indicarle la interfaz de red a usar. Para ver las interfaces de red, por ejemplo, podemos usar el comando *netstat -i*:

<img src="img/netstat_i.png" style="float:left">
<div style="clear:both"></div><br>

Y después usar p0f indicando la interfaz que queremos usar:

<img src="img/p0f_1.png" style="float:left">
<div style="clear:both"></div><br>

Como es una herramienta que captura tráfico se quedará a la escucha. Ahora podemos, por ejemplo, navegar a la dirección web del equipo que queramos saber el sistema operativo:

<img src="img/p0f_2.png" style="float:left">
<div style="clear:both"></div><br>

y localizar en las capturas que hace p0f el sistema operativo:

<img src="img/p0f_3.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo agresivo con nmap

Podemos pasarle a Nmap la opción *-A* para que haga un escaneo agresivo. Es decir, activará las siguientes opciones:

  * Detección de sistema operativo
  * Detección de versiones de los servicios (programas) en cada puerto
  * Ejecutará scripts para analizar vulnerabilidades en los servicios
  * Activará traceroute

Esta opción *-A* suele pasarse a nmap junto con un timing agresivo. El timing quiere decir cuánto tiempo va a esperar nmap a que el servidor le responda por cada petición que haga. El valor más bajo es 0 (esperará mucho tiempo) y el valor más alto es 5 (casi no esperará). En este caso usamos la opción *-T* para pasarle un valor de timing:

<img src="img/nmap_a_scan.png" style="float:left">
<div style="clear:both"></div><br>

Aunque hace mucho ruido, los resultados son muy completos. Por ejemplo, se detecta que el servicio ftp tiene habilitado el acceso anónimo. Aunque este timing hace que tarde poco el scan, se recomienda usar valores de timing más bajos para hacer menos ruido.

#### Escaneo ACK para descubrir dispositivos de seguridad

De acuerdo a la especificación del protocolo [TCP](https://tools.ietf.org/html/rfc793), si enviamos un paquete con la flag ACK activada, deberiamos recibir siempre respuesta tanto si el puerto esta abierto como cerrado. Podemos aprovecharnos de esto para intentar detectar dispositivos de seguridad como ciertos tipos de firewall. Es decir, como siempre que enviamos un paquete con la flag ACK activada, deberiamos recibir respuesta tanto si el puerto esta abierto como si no lo está. Si enviamos un paquete con la flag ACK activada y no recibimos respuesta, probablemente, exista algún dispositivo de seguridad. Para lanzar un escaneo ACK con nmap usamos la opción *-sA*:

<img src="img/nmap_sa_scan.png" style="float:left">
<div style="clear:both"></div><br>

También podemos usar hping3 para lanzar un escaneo ACK pasandole activando la flag ACK con la opción *-A*:

<img src="img/hping3_a.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo con nmap a través de TOR usando proxychains

Si en una auditoria nos bloquean la IP que estemos usando o si simplemente queremos un método sencillo para cambiar la IP desde la que estemos escaneando, podemos usar proxychains para redirigir el tráfico de nuestro equipo a través de una cadena de proxies. En este caso usaremos proxychains para redirigir el tráfico de nuestra máquina a través de la red [TOR](https://en.wikipedia.org/wiki/Tor_%28anonymity_network%29) para cambiar de IP y hacer el escaneo de forma más "anónima". Para ello primero instalamos tor:

<img src="img/nmap_tor_scan_1.png" style="float:left">
<div style="clear:both"></div><br>

y después, si no viniese por defecto instalado, instalamos proxychains:

<img src="img/nmap_tor_scan_2.png" style="float:left">
<div style="clear:both"></div><br>

Si durante la instalación diese algún problema de dependencias, lo solucionamos como de costumbre con:

<img src="img/nmap_tor_scan_3.png" style="float:left">
<div style="clear:both"></div><br>

Una vez instalado, iniciamos el servicio:

<img src="img/nmap_tor_scan_4.png" style="float:left">
<div style="clear:both"></div><br>

Por defecto *proxychains* redirige el tráfico a través de la red TOR. Probamos que la instalación se ha realizado correctamente. Para ello verificamos la IP pública que tenemos, por ejemplo, usando el programa *curl*:

<img src="img/nmap_tor_scan_5.png" style="float:left">
<div style="clear:both"></div><br>

y la que tenemos tras redigir el tráfico a través de TOR usando proxychains:

<img src="img/nmap_tor_scan_6.png" style="float:left">
<div style="clear:both"></div><br>

Podemos usar proxychains con cualquier programa. Por ejemplo, si lo usamos con el navegador firefox:

<img src="img/firefox_tor_1.png" style="float:left">
<div style="clear:both"></div><br>

redirigiremos todo el tráfico a través de TOR y tendremos una nueva IP:

<img src="img/firefox_tor_2.png" style="float:left">
<div style="clear:both"></div><br>

Ahora podemos lanzar nmap de forma "anónima" redirigiendo el tráfico a través de TOR:

<img src="img/nmap_tor_scan_7.png" style="float:left">
<div style="clear:both"></div><br>